Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### More Categorical Encodings

**1.** The article **[Categorical Features and Encoding in Decision Trees](https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931)** mentions 4 encodings:

- **"Categorical Encoding":** This means using the raw categorical values as-is, not encoded. Scikit-learn doesn't support this, but some tree algorithm implementations do. For example, [Catboost](https://catboost.ai/), or R's [rpart](https://cran.r-project.org/web/packages/rpart/index.html) package.
- **Numeric Encoding:** Synonymous with Label Encoding, or "Ordinal" Encoding with random order. We can use [category_encoders.OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html).
- **One-Hot Encoding:** We can use [category_encoders.OneHotEncoder](http://contrib.scikit-learn.org/categorical-encoding/onehot.html).
- **Binary Encoding:** We can use [category_encoders.BinaryEncoder](http://contrib.scikit-learn.org/categorical-encoding/binary.html).


**2.** The short video 
**[Coursera — How to Win a Data Science Competition: Learn from Top Kagglers — Concept of mean encoding](https://www.coursera.org/lecture/competitive-data-science/concept-of-mean-encoding-b5Gxv)** introduces an interesting idea: use both X _and_ y to encode categoricals.

Category Encoders has multiple implementations of this general concept:

- [CatBoost Encoder](http://contrib.scikit-learn.org/categorical-encoding/catboost.html)
- [James-Stein Encoder](http://contrib.scikit-learn.org/categorical-encoding/jamesstein.html)
- [Leave One Out](http://contrib.scikit-learn.org/categorical-encoding/leaveoneout.html)
- [M-estimate](http://contrib.scikit-learn.org/categorical-encoding/mestimate.html)
- [Target Encoder](http://contrib.scikit-learn.org/categorical-encoding/targetencoder.html)
- [Weight of Evidence](http://contrib.scikit-learn.org/categorical-encoding/woe.html)

Category Encoder's mean encoding implementations work for regression problems or binary classification problems. 

For multi-class classification problems, you will need to temporarily reformulate it as binary classification. For example:

```python
encoder = ce.TargetEncoder(min_samples_leaf=..., smoothing=...) # Both parameters > 1 to avoid overfitting
X_train_encoded = encoder.fit_transform(X_train, y_train=='functional')
X_val_encoded = encoder.transform(X_train, y_val=='functional')
```

**3.** The **[dirty_cat](https://dirty-cat.github.io/stable/)** library has a Target Encoder implementation that works with multi-class classification.

```python
 dirty_cat.TargetEncoder(clf_type='multiclass-clf')
```
It also implements an interesting idea called ["Similarity Encoder" for dirty categories](https://www.slideshare.net/GaelVaroquaux/machine-learning-on-non-curated-data-154905090).

However, it seems like dirty_cat doesn't handle missing values or unknown categories as well as category_encoders does. And you may need to use it with one column at a time, instead of with your whole dataframe.

**4. [Embeddings](https://www.kaggle.com/learn/embeddings)** can work well with sparse / high cardinality categoricals.

_**I hope it’s not too frustrating or confusing that there’s not one “canonical” way to encode categorcals. It’s an active area of research and experimentation! Maybe you can make your own contributions!**_

### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [2]:
import os, sys
in_colab = 'google.colab' in sys.modules

# If you're in Colab...
if in_colab:
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
    !git pull origin master
    
    # Install required python packages
    !pip install -r requirements.txt
    
    # Change into directory for module
    os.chdir('module3')

In [3]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

In [11]:
# import block
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('dark_background')
import numpy as np
from scipy.stats import randint, uniform
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
import category_encoders as ce

In [5]:
# train validation split
train, val = train_test_split(train, train_size=0.80, test_size=0.20, 
                              stratify=train['status_group'], random_state=42)

train.shape, val.shape, test.shape

((47520, 41), (11880, 41), (14358, 40))

In [7]:
# Cleaning/engineering function
def wrangler(X):
    
    # Make a copy to avoid warning, prevent making changes from view.
    X = X.copy()
    
    # Replace near-zero latitudes with zero
    X['latitude'] = X['latitude'].replace(-2e-08, 0)
    
    # Replace near-zero longitudes with zero
    X['longitude'] = X['longitude'].replace(-2e-08, 0)
    
    # Swap zeros with nulls
    cols_with_zeros = ['longitude', 'latitude', 'construction_year', 
                       'gps_height', 'population']
    zeros = [0, '0']
    for col in cols_with_zeros:
        X[col] = X[col].replace(zeros, np.nan)
        X[col+'_MISSING'] = X[col].isna()
        
        
    # clean text columns by lowercasing, swapping unknowns with NaNs and add a 'MISSING' column for each
    textcols = ['installer','funder','wpt_name','basin','subvillage','region','lga','ward',
               'scheme_management','scheme_name','extraction_type','extraction_type_group',
               'extraction_type_class','management','management_group','payment','water_quality',
               'quality_group','quantity','source','source_type','source_class','waterpoint_type',
               'waterpoint_type_group']
    unknowns = ['unknown', 'not known', 'none', 'nan', '-', '##', 
          'unknown installer']
    for col in textcols:
        X[col] = X[col].str.lower().str.replace(' ','').str.replace('.','').str.replace('-','')
        X[col] = X[col].replace(unknowns, np.nan)
        X[col+'_MISSING'] = X[col].isna()
        
    # clean boolean columns by imputing the most common value and flagging missing vals
    boolcols = ['public_meeting','permit']
    for col in boolcols:
        X[col+'_MISSING'] = X[col].isna()
    
    # Drop recorded_by (never varies) and id (always varies, random)
    unusable_variance = ['recorded_by', 'id','num_private','wpt_name']
    X = X.drop(columns=unusable_variance)
    
    # create a distance feature for population centers
    X['dardistance'] = (((X['latitude']-(6.7924))**2)+((X['longitude']-(39.2083))**2))**0.5
    X['mwanzadistance'] = (((X['latitude']-(2.5164))**2)+((X['longitude']-(32.9175))**2))**0.5
    X['dodomadistance'] = (((X['latitude']-(6.1630))**2)+((X['longitude']-(35.7516))**2))**0.5
    
    X['dardistance_MISSING'] = X['dardistance'].isnull()
    X['mwanzadistance_MISSING'] = X['mwanzadistance'].isnull()
    X['dodomadistance_MISSING'] = X['dodomadistance'].isnull()
    
    # change date_recorded to datetime format
    X['date_recorded'] = pd.to_datetime(X.date_recorded, infer_datetime_format=True)
    X['date_recorded_MISSING'] = X['date_recorded'].isnull()
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # make list of columns of numeric and categoric type
    numericcolumns = X.select_dtypes(include = 'number').columns.tolist()
    nonnumericcolumns = X.select_dtypes(exclude = 'number').columns.tolist()
    
    # create 'structspect_interval' - number of years between construction and date recorded
    X['structspect_interval'] = X['year_recorded'] - X['construction_year']
    X['structspect_MISSING'] = X['structspect_interval'].isnull()
    
    return X

In [8]:
# Clean and engineer all datasets
train = wrangler(train)
val = wrangler(val)
test = wrangler(test)

In [9]:
# Arrange data into X features matrix and y target vector
target = 'status_group'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

In [10]:
# Mapped version of targets for some encoders
mapdict = {
    'functional': 2,
    'non functional': 0,
    'functional needs repair': 1
}
y_train_mapped = y_train.map(mapdict)
y_val_mapped = y_val.map(mapdict)

In [13]:
# Randomized search with a catboost encoder, sigma adds noise to reduce overfitting
pipeline = make_pipeline(
    ce.cat_boost.CatBoostEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(random_state=42)
)

param_distributions = {
    'catboostencoder__sigma': uniform(0, 5), 
    'simpleimputer__strategy': ['mean', 'median'], 
    'randomforestclassifier__n_estimators': randint(50, 500), 
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None], 
    'randomforestclassifier__max_features': uniform(0, 1), 
}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train_mapped);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  19 out of  30 | elapsed:  6.6min remaining:  3.8min
[Parallel(n_jobs=-1)]: Done  23 out of  30 | elapsed:  8.4min remaining:  2.6min
[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:  8.9min remaining:   59.6s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  9.2min finished


In [14]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

Best hyperparameters {'catboostencoder__sigma': 1.2381539408854136, 'randomforestclassifier__max_depth': 15, 'randomforestclassifier__max_features': 0.07639132578796859, 'randomforestclassifier__n_estimators': 464, 'simpleimputer__strategy': 'median'}
Cross-validation MAE 0.4600378787878788


In [16]:
thelorax = make_pipeline(
    ce.cat_boost.CatBoostEncoder(sigma = 1.23815), 
    SimpleImputer(strategy = 'median'), 
    RandomForestClassifier(random_state=42, max_depth = 42, max_features = 0.07639,
                          n_estimators = 464)
)

thelorax.fit(X_train, y_train_mapped)

Pipeline(memory=None,
         steps=[('catboostencoder',
                 CatBoostEncoder(cols=['funder', 'installer', 'basin',
                                       'subvillage', 'region', 'lga', 'ward',
                                       'public_meeting', 'scheme_management',
                                       'scheme_name', 'permit',
                                       'extraction_type',
                                       'extraction_type_group',
                                       'extraction_type_class', 'management',
                                       'management_group', 'payment',
                                       'payment_type', 'water_quality',
                                       'quality_group', 'q...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=42,
                                        max_features=0.07639,
                                      

In [17]:
thelorax.score(X_val,y_val_mapped)

0.7537037037037037